In [1]:
import pandas as pd

In [2]:
movies = pd.read_csv("Movies.csv")
ratings = pd.read_csv("Ratings.csv")

In [3]:
movies.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [4]:
ratings.head(10)

,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523
5,1,110,4.0,1217896150
6,1,150,3.0,1217895940
7,1,161,4.0,1217897864
8,1,165,3.0,1217897135
9,1,204,0.5,1217895786


In [5]:
def popularity_recommender(genre, threshold, N):
    # Filter the movies dataset to include only movies of the given genre
    genre_movies = movies[movies['genres'].str.contains(genre, na=False)]
    
    # Merge the movies and ratings datasets on the movieId column
    genre_movies_ratings = pd.merge(genre_movies, ratings, on='movieId')
    
    # Group the merged dataset by movieId and calculate the average rating and number of reviews for each movie
    genre_movies_ratings = genre_movies_ratings.groupby(['movieId', 'title']).agg({'rating':'mean', 'userId':'count'}).reset_index()
    
    # Filter the dataset to include only movies that have at least the given number of reviews
    genre_movies_ratings = genre_movies_ratings[genre_movies_ratings['userId'] >= threshold]
    
    # Sort the dataset by average rating in descending order
    genre_movies_ratings = genre_movies_ratings.sort_values(by='rating', ascending=False)
    
    # Select the top N movies
    top_N = genre_movies_ratings.head(N)
    
    # Add a column to indicate the ranking
    top_N.insert(0, 'S.No', range(1, len(top_N) + 1))
    
    # Return the final dataframe
    return top_N

In [6]:
import ipywidgets as widgets
from IPython.display import display

def popularity_recommender_ui():
    genre_dropdown = widgets.Dropdown(options=movies['genres'].unique())
    threshold_slider = widgets.IntSlider(min=1, max=100, value=10)
    N_slider = widgets.IntSlider(min=1, max=10, value=5)
    run_button = widgets.Button(description="Run")
    output = widgets.Output()

    def on_button_click(b):
        with output:
            output.clear_output()
            display(popularity_recommender(genre_dropdown.value, threshold_slider.value, N_slider.value))

    run_button.on_click(on_button_click)

    display(genre_dropdown, threshold_slider, N_slider, run_button, output)
    
popularity_recommender_ui()

Dropdown(options=('Adventure|Animation|Children|Comedy|Fantasy', 'Adventure|Children|Fantasy', 'Comedy|Romance…

IntSlider(value=10, min=1)

IntSlider(value=5, max=10, min=1)

Button(description='Run', style=ButtonStyle())

Output()